In [12]:
from bokeh.models import BoxZoomTool
import pandas as pd
import numpy as np

In [13]:
import glob
tics = glob.glob('/run/user/1000/gvfs/smb-share:server=silo.physics.usyd.edu.au,share=silo2/dhey3294/TESS/Searches/TESS_pg/res/*.png')

# Some glomming of files and ids
ticids = [tic.split('/res/')[-1].split('.')[0] for tic in tics]
tic_pg = pd.DataFrame({
    'ID': ticids,
    'files': tics
})

In [ ]:
# from tqdm import tqdm

# # We need to chunk up our query because MAST will time-out.. sorry servers!
# chunked_ids = np.array_split(ticids, 100)
# df = Catalogs.query_criteria(catalog="Tic", ID=chunked_ids[0]).to_pandas()
# for chunked_id in tqdm(chunked_ids[1:]):
#     df = df.append(Catalogs.query_criteria(catalog="Tic", ID=chunked_id).to_pandas())

In [14]:
df = pd.read_csv('tic_query.csv', dtype={'ID': str})
# df.to_csv('tic_query.csv')

In [15]:
# # Let's merge our dataframes
# test = df.merge(tic_pg, left_on='ID', right_on='ID')
# mask = (np.isfinite(test['Teff'])) & (np.isfinite(test['lum']))

In [16]:
github_files = [f'https://github.com/danhey/HRDTess/raw/master/data/{ticid}.png' for ticid in ticids]
tic_pg = pd.DataFrame({
    'ID': ticids,
    'files': github_files
})
test = df.merge(tic_pg, left_on='ID', right_on='ID')
mask = (np.isfinite(test['Teff'])) & (np.isfinite(test['lum']))

In [17]:
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.models import HoverTool

output_file("index.html")

source = ColumnDataSource(
        data=dict(
            x=test[mask]['Teff'].values,
            y=np.log10(test[mask]['lum'].values),
            imgs = test[mask]['files'].values,
            desc = test[mask]['ID'].values
        )
    )

hover = HoverTool(# height="200" alt="@imgs" width="500"
        tooltips="""
        <div>
            <div>
                <img
                    src="@imgs"
                    border="2"
                ></img>
            </div>
            <div>
                <span style="font-size: 17px; font-weight: bold;">TIC @desc</span>
            </div>
        </div>
        """
    )

p = figure(#plot_width=900, plot_height=900, 
           tools=[hover, 'zoom_in', 'undo', 'box_zoom', 'reset'],
           title="TESS in the Southern Hemisphere", 
           toolbar_location="above",
           sizing_mode='stretch_both',
           x_range=(12000,2500), #y_range=(bottom, top)
          )

p.circle('x', 'y', 
         radius=5,
#          size=7,
         fill_color="firebrick", 
         line_color='black',
         source=source)

p.xaxis.axis_label = 'Temperature (K)'
p.yaxis.axis_label = r'Log10(L/L⊙)'
p.xaxis.axis_label_text_font_size = "18pt"
p.yaxis.axis_label_text_font_size = "18pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.title.text_font_size = "15pt"

p.xaxis.axis_label_text_font_style = 'normal'
p.yaxis.axis_label_text_font_style = 'normal'

show(p)